# 🏀 NBA Match simulator


Le projet permet de **comparer deux lineups NBA** et de **prédire laquelle est la plus performante dans un match simulé**.
**Lineups aléatoires** composés de 5 joueurs sélectionnés individuellement


# Importation des données

In [5]:
import pandas as pd

df_v1=pd.read_csv('../Data/df_v1.csv')
df_draft_combine=pd.read_csv("../Data/df_draft_combine_cleaned.csv")
df_draft_history=pd.read_csv("../Data/df_draft_history_cleaned.csv")
df_game=pd.read_csv("../Data/v1/df_game_cleaned.csv")
games=pd.read_csv("../Data/df_game_info_cleaned.csv")
df_sum =pd.read_csv("../Data/df_game_summary_cleaned.csv")
officials=pd.read_csv("../Data/df_officials_cleaned.csv")
other=pd.read_csv("../Data/df_other_stats_cleaned.csv")

In [32]:

print("Colonnes:", games.columns.tolist())
print("\nAperçu des données:")
print(df_v1.head())

Colonnes: ['game_id', 'game_date', 'attendance', 'game_time']

Aperçu des données:
        player_name team_abbreviation   age  player_height  player_weight  \
0  Randy Livingston               HOU  22.0         193.04      94.800728   
1  Gaylon Nickerson               WAS  28.0         190.50      86.182480   
2      George Lynch               VAN  26.0         203.20     103.418976   
3    George McCloud               LAL  30.0         203.20     102.058200   
4      George Zidek               DEN  23.0         213.36     119.748288   

                 college country draft_year draft_round draft_number  ...  \
0        Louisiana State     USA       1996           2           42  ...   
1  Northwestern Oklahoma     USA       1994           2           34  ...   
2         North Carolina     USA       1993           1           12  ...   
3          Florida State     USA       1989           1            7  ...   
4                   UCLA     USA       1995           1           22 

# Modèle

In [38]:
df_game.columns

Index(['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home',
       'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away',
       'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away',
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away',
       'season_type'],
      dtype='object')

## Modèle 1

In [16]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import random
from datetime import datetime

class LineupPredictor:
    def __init__(self):
        self.players_data = None
        self.games_data = None
        self.team_stats = None
        # Définir les poids des positions selon leur importance
        self.position_weights = {
            'PG': 1.2,  # Point Guard - plus important pour l'organisation
            'SG': 1.1,  # Shooting Guard - important pour le scoring
            'SF': 1.0,  # Small Forward - polyvalent
            'PF': 1.0,  # Power Forward - important pour les rebonds
            'C': 1.1    # Center - crucial pour la défense et les rebonds
        }
        
    def load_data(self):
        """Charge et prépare les données nécessaires"""
        # Charger les données des joueurs
        self.players_data = pd.read_csv('../Data/df_v1.csv')
        
        # Nettoyer les données des joueurs
        self._clean_player_data()
        
        # Calculer les statistiques d'équipe
        self._calculate_team_stats()
        
    def _clean_player_data(self):
        """Nettoie et prépare les données des joueurs"""
        # Convertir la position en format standard
        self.players_data['position'] = self.players_data['position'].fillna('Unknown')
        
        # Remplacer les valeurs manquantes par des moyennes
        numeric_columns = ['pts', 'reb', 'ast', 'net_rating', 'oreb_pct', 'dreb_pct', 
                         'usg_pct', 'ts_pct', 'ast_pct', 'win_rate']
        
        for col in numeric_columns:
            if col in self.players_data.columns:
                self.players_data[col] = self.players_data[col].fillna(self.players_data[col].mean())
        
        # Créer des catégories de position
        def categorize_position(pos):
            if pd.isna(pos) or pos == 'Unknown':
                return random.choice(['PG', 'SG', 'SF', 'PF', 'C'])
            pos = str(pos).upper()
            if 'GUARD' in pos:
                return 'PG' if random.random() < 0.5 else 'SG'
            elif 'FORWARD' in pos:
                return 'SF' if random.random() < 0.5 else 'PF'
            elif 'CENTER' in pos or 'C' in pos:
                return 'C'
            return random.choice(['PG', 'SG', 'SF', 'PF', 'C'])
        
        self.players_data['position'] = self.players_data['position'].apply(categorize_position)
        
    def _calculate_team_stats(self):
        """Calcule les statistiques d'équipe basées sur les données des joueurs"""
        self.team_stats = self.players_data.groupby('team_abbreviation').agg({
            'win_rate': 'mean',
            'pts': 'mean',
            'reb': 'mean',
            'ast': 'mean',
            'net_rating': 'mean',
            'ts_pct': 'mean'
        }).reset_index()
        
    def select_random_lineup(self) -> List[Dict]:
        """Sélectionne aléatoirement une lineup de 5 joueurs avec des positions équilibrées"""
        # Définir les positions requises
        required_positions = ['PG', 'SG', 'SF', 'PF', 'C']
        lineup = []
        
        # Sélectionner un joueur pour chaque position
        for position in required_positions:
            # Filtrer les joueurs pour cette position
            position_players = self.players_data[self.players_data['position'] == position]
            
            if len(position_players) > 0:
                # Sélectionner un joueur aléatoire
                player = position_players.sample(n=1).iloc[0]
                
                lineup.append({
                    'player_id': player.get('person_id'),
                    'name': player['player_name'],
                    'position': position,
                    'team': player['team_abbreviation'],
                    'stats': {
                        'pts': player['pts'],
                        'reb': player['reb'],
                        'ast': player['ast'],
                        'net_rating': player['net_rating'],
                        'oreb_pct': player['oreb_pct'],
                        'dreb_pct': player['dreb_pct'],
                        'usg_pct': player['usg_pct'],
                        'ts_pct': player['ts_pct'],
                        'ast_pct': player['ast_pct']
                    }
                })
        
        return lineup
    
    def calculate_lineup_score(self, lineup: List[Dict]) -> float:
        """Calcule le score d'une lineup basé sur les statistiques des joueurs et de l'équipe"""
        total_score = 0
        
        # Statistiques de l'équipe
        team_stats = self.team_stats[self.team_stats['team_abbreviation'] == lineup[0]['team']].iloc[0]
        team_score = (
            team_stats['win_rate'] * 50 +  # Le winrate de l'équipe est très important
            team_stats['net_rating'] * 2 +  # Le rating net de l'équipe
            team_stats['ts_pct'] * 10      # Le pourcentage de tir réel
        )
        
        # Score des joueurs
        for player in lineup:
            # Score de base basé sur les statistiques
            player_score = (
                player['stats']['pts'] * 1.0 +      # Points marqués
                player['stats']['reb'] * 0.8 +      # Rebonds
                player['stats']['ast'] * 0.8 +      # Passes décisives
                player['stats']['net_rating'] * 0.5 + # Rating net
                player['stats']['oreb_pct'] * 5 +   # Pourcentage de rebonds offensifs
                player['stats']['dreb_pct'] * 5 +   # Pourcentage de rebonds défensifs
                player['stats']['usg_pct'] * 3 +    # Pourcentage d'utilisation
                player['stats']['ts_pct'] * 5 +     # Pourcentage de tir réel
                player['stats']['ast_pct'] * 3      # Pourcentage de passes décisives
            )
            
            # Appliquer le poids de la position
            position_weight = self.position_weights[player['position']]
            total_score += player_score * position_weight
        
        # Combiner le score des joueurs avec le score de l'équipe
        return total_score + team_score
    
    def predict_winner(self, lineup1: List[Dict], lineup2: List[Dict]) -> Tuple[List[Dict], float]:
        """Prédit le gagnant entre deux lineups"""
        score1 = self.calculate_lineup_score(lineup1)
        score2 = self.calculate_lineup_score(lineup2)
        
        if score1 > score2:
            return lineup1, score1 - score2
        else:
            return lineup2, score2 - score1

# Exemple d'utilisation
if __name__ == "__main__":
    predictor = LineupPredictor()
    predictor.load_data()
    
    # Sélectionner deux lineups aléatoires
    lineup1 = predictor.select_random_lineup()
    lineup2 = predictor.select_random_lineup()
    
    # Prédire le gagnant
    winner, margin = predictor.predict_winner(lineup1, lineup2)
    
    print("Lineup 1:")
    for player in lineup1:
        print(f"{player['position']}: {player['name']} ({player['team']})")
    
    print("\nLineup 2:")
    for player in lineup2:
        print(f"{player['position']}: {player['name']} ({player['team']})")
    
    print("\nGagnant prédit:")
    for player in winner:
        print(f"{player['position']}: {player['name']} ({player['team']})")
    print(f"Marge de victoire prédite: {margin:.2f}") 

Lineup 1:
PG: Brandon Goodwin (CLE)
SG: Mike Muscala (BOS)
SF: Pau Gasol (LAL)
PF: Brandon Clarke (MEM)
C: Abdel Nader (BOS)

Lineup 2:
PG: Aleksandar Djordjevic (POR)
SG: Raul Neto (WAS)
SF: Cody Martin (CHA)
PF: Darren Collison (LAL)
C: Etan Thomas (WAS)

Gagnant prédit:
PG: Brandon Goodwin (CLE)
SG: Mike Muscala (BOS)
SF: Pau Gasol (LAL)
PF: Brandon Clarke (MEM)
C: Abdel Nader (BOS)
Marge de victoire prédite: 61.14


## Modèle 2

In [30]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import random
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

class LineupPredictor:
    def __init__(self):
        self.players_data = None
        self.draft_combine_data = None
        self.team_stats = None
        self.model = None
        self.features = None
        self.scaler = StandardScaler()
        
    def load_data(self):
        """Charge et prépare les données nécessaires"""
        # Charger les données des joueurs
        self.players_data = pd.read_csv('../Data/df_v1.csv')
        
        # Charger les données du draft combine
        self.draft_combine_data = pd.read_csv('../Data/df_draft_combine_cleaned.csv')
        
        # Convertir les saisons au même format
        self.players_data['season_year'] = self.players_data['season'].str[:4].astype(int)
        
        # Nettoyer et préparer les données
        self._clean_and_prepare_data()
        
        # Calculer les statistiques d'équipe
        self._calculate_team_stats()
        
        # Entraîner le modèle
        self._train_model()
        
    def _clean_and_prepare_data(self):
        """Nettoie et prépare les données des joueurs"""
        # Feature engineering
        self.players_data['pts_per_game'] = self.players_data['pts'] / self.players_data['gp']
        self.players_data['reb_per_game'] = self.players_data['reb'] / self.players_data['gp']
        self.players_data['ast_per_game'] = self.players_data['ast'] / self.players_data['gp']
        self.players_data['ast_usg_ratio'] = self.players_data['ast_pct'] / (self.players_data['usg_pct'] + 1e-6)
        self.players_data['reb_pct_sum'] = self.players_data['oreb_pct'] + self.players_data['dreb_pct']
        
        # Gestion des positions
        def get_primary_pos(pos):
            if pd.isna(pos):
                return np.nan
            pos = str(pos).upper()
            if "GUARD" in pos and "CENTER" not in pos:
                return "G"
            if "CENTER" in pos:
                return "C"
            if "FORWARD" in pos:
                return "F"
            return np.nan
        
        self.players_data['primary_pos'] = self.players_data['position'].apply(get_primary_pos)
        
        # Fusionner avec les données du draft combine
        if 'player_name' in self.draft_combine_data.columns:
            # Sélectionner les colonnes pertinentes du draft combine
            combine_features = [
                'player_name', 'height_wo_shoes_cm', 'wingspan_cm', 
                'standing_reach_cm', 'body_fat_pct', 'standing_vertical_leap',
                'max_vertical_leap', 'lane_agility_time', 'three_quarter_sprint'
            ]
            
            combine_data = self.draft_combine_data[combine_features].copy()
            
            # Fusionner sur le nom du joueur
            self.players_data = pd.merge(
                self.players_data,
                combine_data,
                on='player_name',
                how='left'
            )
        
        # Remplacer les valeurs manquantes
        numeric_columns = [
            'pts_per_game', 'reb_per_game', 'ast_per_game',
            'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct',
            'net_rating', 'ast_usg_ratio', 'reb_pct_sum', 'win_rate'
        ]
        
        for col in numeric_columns:
            if col in self.players_data.columns:
                self.players_data[col] = self.players_data[col].fillna(self.players_data[col].mean())
        
    def _calculate_team_stats(self):
        """Calcule les statistiques d'équipe"""
        self.team_stats = self.players_data.groupby('team_abbreviation').agg({
            'win_rate': 'mean',
            'pts_per_game': 'mean',
            'reb_per_game': 'mean',
            'ast_per_game': 'mean',
            'net_rating': 'mean',
            'ts_pct': 'mean',
            'ast_usg_ratio': 'mean',
            'reb_pct_sum': 'mean'
        }).reset_index()
        
    def _train_model(self):
        """Entraîne le modèle de prédiction"""
        # Définir les features pour le modèle
        features_all = [
            'pts_per_game', 'reb_per_game', 'ast_per_game',
            'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct',
            'net_rating', 'ast_usg_ratio', 'reb_pct_sum'
        ]
        
        # Calculer les corrélations avec win_rate
        corr = self.players_data[features_all + ['win_rate']].corr()
        corr_win = corr['win_rate'].abs().drop('win_rate').sort_values(ascending=False)
        
        # Sélectionner les 5 meilleures features
        self.features = corr_win.index.tolist()[:5]
        
        # Préparer les données d'entraînement
        train_data = self.players_data.groupby(['season_year', 'team_abbreviation']).apply(
            lambda x: x.nlargest(5, 'pts')
        ).reset_index(drop=True)
        
        # Agréger les statistiques par équipe
        team_stats = train_data.groupby(['season_year', 'team_abbreviation'])[self.features].mean().reset_index()
        
        # Fusionner avec le winrate
        team_stats = pd.merge(
            team_stats,
            self.players_data[['season_year', 'team_abbreviation', 'win_rate']].drop_duplicates(),
            on=['season_year', 'team_abbreviation']
        )
        
        # Entraîner le modèle
        X = team_stats[self.features]
        y = team_stats['win_rate']
        
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model.fit(X, y)
        
    def get_lineup_coherence(self, lineup: List[Dict]) -> Tuple[float, str]:
        """Calcule la cohérence d'un lineup et retourne le bonus et la description"""
        positions = [player['position'] for player in lineup]
        unique_positions = set(positions)
        
        # Vérifier la présence de chaque position
        has_guard = 'G' in unique_positions
        has_forward = 'F' in unique_positions
        has_center = 'C' in unique_positions
        
        # Compter le nombre de positions uniques
        num_unique_positions = len(unique_positions)
        
        # Définir les bonus selon la cohérence
        if num_unique_positions == 3 and positions.count('G') == 2 and positions.count('F') == 2 and positions.count('C') == 1:
            return 1.15, "Lineup parfaite (2G, 2F, 1C)"
        elif num_unique_positions == 3:
            return 1.10, "Lineup avec les 3 positions (G, F, C)"
        elif num_unique_positions == 2:
            if has_guard and has_forward:
                return 1.05, "Lineup avec Guards et Forwards"
            elif has_guard and has_center:
                return 1.05, "Lineup avec Guards et Center"
            elif has_forward and has_center:
                return 1.05, "Lineup avec Forwards et Center"
        elif num_unique_positions == 1:
            return 0.95, "Lineup avec une seule position"
        
        return 1.0, "Lineup standard"
        
    def select_random_lineup(self) -> List[Dict]:
        """Sélectionne aléatoirement une lineup de 5 joueurs"""
        # Sélectionner 5 joueurs au hasard parmi les meilleurs
        top_players = self.players_data.nlargest(100, 'net_rating')
        selected_players = top_players.sample(n=5)
        
        lineup = []
        for _, player in selected_players.iterrows():
            stats = {
                'pts_per_game': player['pts_per_game'],
                'reb_per_game': player['reb_per_game'],
                'ast_per_game': player['ast_per_game'],
                'net_rating': player['net_rating'],
                'oreb_pct': player['oreb_pct'],
                'dreb_pct': player['dreb_pct'],
                'usg_pct': player['usg_pct'],
                'ts_pct': player['ts_pct'],
                'ast_pct': player['ast_pct'],
                'ast_usg_ratio': player['ast_usg_ratio'],
                'reb_pct_sum': player['reb_pct_sum']
            }
            
            lineup.append({
                'player_id': player.get('person_id'),
                'name': player['player_name'],
                'position': player['primary_pos'],
                'team': player['team_abbreviation'],
                'stats': stats
            })
        
        return lineup
    
    def calculate_lineup_score(self, lineup: List[Dict]) -> Tuple[float, str]:
        """Calcule le score d'une lineup basé sur les statistiques des joueurs et de l'équipe"""
        # Calculer les statistiques moyennes de la lineup
        lineup_stats = pd.DataFrame([player['stats'] for player in lineup])
        avg_stats = lineup_stats[self.features].mean()
        
        # Prédire le winrate avec le modèle
        predicted_winrate = self.model.predict(avg_stats.values.reshape(1, -1))[0]
        
        # Calculer le bonus de cohérence
        coherence_bonus, coherence_desc = self.get_lineup_coherence(lineup)
        predicted_winrate *= coherence_bonus
        
        return predicted_winrate, coherence_desc
    
    def predict_winner(self, lineup1: List[Dict], lineup2: List[Dict]) -> Tuple[List[Dict], float, float, str, str]:
        """Prédit le gagnant entre deux lineups"""
        score1, desc1 = self.calculate_lineup_score(lineup1)
        score2, desc2 = self.calculate_lineup_score(lineup2)
        
        if score1 > score2:
            return lineup1, score1, score2, desc1, desc2
        else:
            return lineup2, score2, score1, desc2, desc1

# Exemple d'utilisation
if __name__ == "__main__":
    predictor = LineupPredictor()
    predictor.load_data()
    
    # Sélectionner deux lineups aléatoires
    lineup1 = predictor.select_random_lineup()
    lineup2 = predictor.select_random_lineup()
    
    # Prédire le gagnant
    winner, winner_score, loser_score, winner_desc, loser_desc = predictor.predict_winner(lineup1, lineup2)
    
    print("Lineup 1:")
    for player in lineup1:
        print(f"{player['position']}: {player['name']} ({player['team']})")
    print(f"Winrate prédit: {winner_score if winner == lineup1 else loser_score:.3f}")
    print(f"Description: {winner_desc if winner == lineup1 else loser_desc}")
    
    print("\nLineup 2:")
    for player in lineup2:
        print(f"{player['position']}: {player['name']} ({player['team']})")
    print(f"Winrate prédit: {winner_score if winner == lineup2 else loser_score:.3f}")
    print(f"Description: {winner_desc if winner == lineup2 else loser_desc}")
    
    print("\nGagnant prédit:")
    for player in winner:
        print(f"{player['position']}: {player['name']} ({player['team']})")
    print(f"Winrate prédit: {winner_score:.3f}")
    print(f"Description: {winner_desc}")

Lineup 1:
F: Tim James (MIA)
nan: Tyson Wheeler (DEN)
nan: Tyler Zeller (SAS)
C: Amida Brimah (IND)
F: Udonis Haslem (MIA)
Winrate prédit: 0.773
Description: Lineup avec les 3 positions (G, F, C)

Lineup 2:
F: Slava Medvedenko (LAL)
nan: Von Wafer (LAC)
G: Dereon Seabron (NOP)
nan: Deonte Burton (SAC)
nan: Shavlik Randolph (PHI)
Winrate prédit: 0.892
Description: Lineup avec les 3 positions (G, F, C)

Gagnant prédit:
F: Slava Medvedenko (LAL)
nan: Von Wafer (LAC)
G: Dereon Seabron (NOP)
nan: Deonte Burton (SAC)
nan: Shavlik Randolph (PHI)
Winrate prédit: 0.892
Description: Lineup avec les 3 positions (G, F, C)


/var/folders/t6/j8g_7ls97b9gdfqfkc7p75q80000gn/T/ipykernel_64614/3679884524.py:122: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data = self.players_data.groupby(['season_year', 'team_abbreviation']).apply(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


## Modèle final (on ajoute les games)

### Feature Engineering

Le système calcule plusieurs métriques avancées pour chaque joueur :

### 1. Statistiques par match :

- `pts_per_game`, `reb_per_game`, `ast_per_game`

### 2. Métriques d'efficacité :

- `efficiency` = `(pts + reb + ast) / gp`
- `scoring_efficiency` = `pts_per_game / usg_pct`
- `playmaking` = `ast_per_game * ast_pct`
- `ast_usg_ratio` = `ast_pct / usg_pct`
- `reb_pct_sum` = `oreb_pct + dreb_pct`

### 3. Classification des positions :

- `G` : Guard
- `F` : Forward
- `C` : Center

### Modèle de prédiction

Un modèle `RandomForestRegressor` est entraîné pour prédire le **taux de victoire (`win_rate`)** d’un lineup à partir des statistiques agrégées de ses joueurs.

- Sélection des 8 features les plus corrélées avec `win_rate`
- Agrégation des statistiques des 5 meilleurs scoreurs de chaque équipe
- Entraînement du modèle : 200 arbres, profondeur maximale de 10

### Systèmes de sélection de lineup

`select_random_lineup()` : sélectionne aléatoirement 5 joueurs parmi tout l'historique NBA

Les statistiques individuelles sont récupérées et les positions des joueurs sont utilisées pour évaluer la cohérence de la lineup.

### Système de bonus de cohérence

Un bonus est appliqué au taux de victoire prédit en fonction de la distribution des postes :

- **+15%** pour une lineup idéale (2 Guards, 2 Forwards, 1 Center)
- **+10%** pour une lineup avec les 3 positions représentées
- **+5%** pour une lineup avec 2 types de positions
- **−5%** pour une lineup composée d’un seul type de poste

### Procédure de prédiction

1. Sélection de deux lineups (aléatoires ou réelles)
2. Calcul des statistiques agrégées
3. Prédiction du `win_rate` via le modèle RandomForest
4. Application du bonus de cohérence
5. Comparaison des scores finaux
6. Affichage détaillé des joueurs et du score du lineup gagnant

In [42]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import random
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

class LineupPredictor:
    def __init__(self):
        self.players_data = None
        self.draft_combine_data = None
        self.game_data = None
        self.team_stats = None
        self.model = None
        self.features = None
        self.scaler = StandardScaler()
        
    def load_data(self):
        """Charge et prépare les données nécessaires"""
        # Charger les données des joueurs
        self.players_data = pd.read_csv('../Data/df_v1.csv')
        
        # Charger les données du draft combine (uniquement pour l'affichage)
        self.draft_combine_data = pd.read_csv('../Data/df_draft_combine_cleaned.csv')
        
        # Charger les données des matchs
        self.game_data = pd.read_csv('../Data/df_game_summary_cleaned.csv')
        
        # Convertir les saisons au même format et filtrer après 2000
        self.players_data['season_year'] = self.players_data['season'].str[:4].astype(int)
        self.players_data = self.players_data[self.players_data['season_year'] >= 2000]
        
        # Nettoyer et préparer les données
        self._clean_and_prepare_data()
        
        # Calculer les statistiques d'équipe
        self._calculate_team_stats()
        
        # Entraîner le modèle
        self._train_model()
        
    def _clean_and_prepare_data(self):
        """Nettoie et prépare les données des joueurs"""
        # Feature engineering pour les joueurs
        self.players_data['pts_per_game'] = self.players_data['pts'] / self.players_data['gp']
        self.players_data['reb_per_game'] = self.players_data['reb'] / self.players_data['gp']
        self.players_data['ast_per_game'] = self.players_data['ast'] / self.players_data['gp']
        self.players_data['ast_usg_ratio'] = self.players_data['ast_pct'] / (self.players_data['usg_pct'] + 1e-6)
        self.players_data['reb_pct_sum'] = self.players_data['oreb_pct'] + self.players_data['dreb_pct']
        
        # Ajouter des statistiques avancées
        self.players_data['efficiency'] = (self.players_data['pts'] + self.players_data['reb'] + self.players_data['ast']) / self.players_data['gp']
        self.players_data['scoring_efficiency'] = self.players_data['pts_per_game'] / (self.players_data['usg_pct'] + 1e-6)
        self.players_data['playmaking'] = self.players_data['ast_per_game'] * self.players_data['ast_pct']
        
        # Gestion des positions
        def get_primary_pos(pos):
            if pd.isna(pos):
                return np.nan
            pos = str(pos).upper()
            if "GUARD" in pos and "CENTER" not in pos:
                return "G"
            if "CENTER" in pos:
                return "C"
            if "FORWARD" in pos:
                return "F"
            return np.nan
        
        self.players_data['primary_pos'] = self.players_data['position'].apply(get_primary_pos)
        
        # Préparer les données des matchs
        if self.game_data is not None:
            # Créer deux dataframes séparés pour les équipes à domicile et à l'extérieur
            home_games = self.game_data.copy()
            visitor_games = self.game_data.copy()
            
            # Renommer les colonnes pour la fusion
            home_games = home_games.rename(columns={'home_team_id': 'team_id'})
            visitor_games = visitor_games.rename(columns={'visitor_team_id': 'team_id'})
            
            # Combiner les deux dataframes
            all_games = pd.concat([home_games, visitor_games], ignore_index=True)
            
            # Calculer les statistiques moyennes par équipe et par saison
            game_stats = all_games.groupby(['season', 'team_id']).agg({
                'game_id': 'count'  # Nombre de matchs joués
            }).reset_index()
            
            # Renommer la colonne de compte
            game_stats = game_stats.rename(columns={'game_id': 'games_played'})
            
            # Fusionner avec les données des joueurs
            # Note: Nous devons d'abord nous assurer que team_id correspond à team_abbreviation
            # Pour l'instant, nous allons simplement ajouter les statistiques de base
            self.players_data['games_played'] = self.players_data['gp']
        
        # Remplacer les valeurs manquantes
        numeric_columns = [
            'pts_per_game', 'reb_per_game', 'ast_per_game',
            'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct',
            'net_rating', 'ast_usg_ratio', 'reb_pct_sum', 'win_rate',
            'efficiency', 'scoring_efficiency', 'playmaking'
        ]
        
        for col in numeric_columns:
            if col in self.players_data.columns:
                self.players_data[col] = self.players_data[col].fillna(self.players_data[col].mean())
        
    def _calculate_team_stats(self):
        """Calcule les statistiques d'équipe"""
        # Statistiques de base
        self.team_stats = self.players_data.groupby('team_abbreviation').agg({
            'win_rate': 'mean',
            'pts_per_game': 'mean',
            'reb_per_game': 'mean',
            'ast_per_game': 'mean',
            'net_rating': 'mean',
            'ts_pct': 'mean',
            'ast_usg_ratio': 'mean',
            'reb_pct_sum': 'mean',
            'efficiency': 'mean',
            'scoring_efficiency': 'mean',
            'playmaking': 'mean'
        }).reset_index()
        
    def _train_model(self):
        """Entraîne le modèle de prédiction"""
        # Définir les features pour le modèle
        features_all = [
            'pts_per_game', 'reb_per_game', 'ast_per_game',
            'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct',
            'net_rating', 'ast_usg_ratio', 'reb_pct_sum',
            'efficiency', 'scoring_efficiency', 'playmaking'
        ]
        
        # Calculer les corrélations avec win_rate
        corr = self.players_data[features_all + ['win_rate']].corr()
        corr_win = corr['win_rate'].abs().drop('win_rate').sort_values(ascending=False)
        
        # Sélectionner les 8 meilleures features
        self.features = corr_win.index.tolist()[:8]
        
        # Préparer les données d'entraînement
        train_data = self.players_data.groupby(['season_year', 'team_abbreviation']).apply(
            lambda x: x.nlargest(5, 'pts')
        ).reset_index(drop=True)
        
        # Agréger les statistiques par équipe
        team_stats = train_data.groupby(['season_year', 'team_abbreviation'])[self.features].mean().reset_index()
        
        # Fusionner avec le winrate
        team_stats = pd.merge(
            team_stats,
            self.players_data[['season_year', 'team_abbreviation', 'win_rate']].drop_duplicates(),
            on=['season_year', 'team_abbreviation']
        )
        
        # Entraîner le modèle
        X = team_stats[self.features]
        y = team_stats['win_rate']
        
        self.model = RandomForestRegressor(n_estimators=200, random_state=42, max_depth=10)
        self.model.fit(X, y)
        
    def get_lineup_coherence(self, lineup: List[Dict]) -> Tuple[float, str]:
        """Calcule la cohérence d'un lineup et retourne le bonus et la description"""
        # Ne considérer que les positions connues (non-NaN)
        positions = [player['position'] for player in lineup if pd.notna(player['position'])]
        if not positions:  # Si aucune position n'est connue
            return 1.0, "Positions inconnues"
            
        unique_positions = set(positions)
        
        # Vérifier la présence de chaque position
        has_guard = 'G' in unique_positions
        has_forward = 'F' in unique_positions
        has_center = 'C' in unique_positions
        
        # Compter le nombre de positions uniques
        num_unique_positions = len(unique_positions)
        
        # Définir les bonus selon la cohérence
        if num_unique_positions == 3 and positions.count('G') == 2 and positions.count('F') == 2 and positions.count('C') == 1:
            return 1.15, "Lineup parfaite (2G, 2F, 1C)"
        elif num_unique_positions == 3:
            return 1.10, "Lineup avec les 3 positions (G, F, C)"
        elif num_unique_positions == 2:
            if has_guard and has_forward:
                return 1.05, "Lineup avec Guards et Forwards"
            elif has_guard and has_center:
                return 1.05, "Lineup avec Guards et Center"
            elif has_forward and has_center:
                return 1.05, "Lineup avec Forwards et Center"
        elif num_unique_positions == 1:
            return 0.95, "Lineup avec une seule position"
        
        return 1.0, "Lineup standard"
        
    def select_random_lineup(self) -> List[Dict]:
        """Sélectionne aléatoirement une lineup de 5 joueurs"""
        # Sélectionner 5 joueurs au hasard parmi les meilleurs
        top_players = self.players_data.nlargest(100, 'efficiency')
        selected_players = top_players.sample(n=5)
        
        lineup = []
        for _, player in selected_players.iterrows():
            stats = {
                'pts_per_game': player['pts_per_game'],
                'reb_per_game': player['reb_per_game'],
                'ast_per_game': player['ast_per_game'],
                'net_rating': player['net_rating'],
                'oreb_pct': player['oreb_pct'],
                'dreb_pct': player['dreb_pct'],
                'usg_pct': player['usg_pct'],
                'ts_pct': player['ts_pct'],
                'ast_pct': player['ast_pct'],
                'ast_usg_ratio': player['ast_usg_ratio'],
                'reb_pct_sum': player['reb_pct_sum'],
                'efficiency': player['efficiency'],
                'scoring_efficiency': player['scoring_efficiency'],
                'playmaking': player['playmaking']
            }
            
            # Ajouter les données du draft combine si disponibles
            combine_data = {}
            if 'player_name' in self.draft_combine_data.columns:
                player_combine = self.draft_combine_data[self.draft_combine_data['player_name'] == player['player_name']]
                if not player_combine.empty:
                    combine_data = {
                        'height': player_combine['height_wo_shoes_cm'].iloc[0],
                        'wingspan': player_combine['wingspan_cm'].iloc[0],
                        'standing_reach': player_combine['standing_reach_cm'].iloc[0],
                        'body_fat': player_combine['body_fat_pct'].iloc[0],
                        'vertical_leap': player_combine['standing_vertical_leap'].iloc[0],
                        'max_vertical': player_combine['max_vertical_leap'].iloc[0],
                        'agility': player_combine['lane_agility_time'].iloc[0],
                        'sprint': player_combine['three_quarter_sprint'].iloc[0]
                    }
            
            lineup.append({
                'player_id': player.get('person_id'),
                'name': player['player_name'],
                'position': player['primary_pos'],
                'team': player['team_abbreviation'],
                'stats': stats,
                'combine_data': combine_data
            })
        
        return lineup
    
    def calculate_lineup_score(self, lineup: List[Dict]) -> Tuple[float, str]:
        """Calcule le score d'une lineup basé sur les statistiques des joueurs et de l'équipe"""
        # Calculer les statistiques moyennes de la lineup
        lineup_stats = pd.DataFrame([player['stats'] for player in lineup])
        avg_stats = lineup_stats[self.features].mean()
        
        # Prédire le winrate avec le modèle
        predicted_winrate = self.model.predict(avg_stats.values.reshape(1, -1))[0]
        
        # Calculer le bonus de cohérence
        coherence_bonus, coherence_desc = self.get_lineup_coherence(lineup)
        
        # Appliquer le bonus de cohérence
        predicted_winrate *= coherence_bonus
        
        # S'assurer que le winrate reste dans des limites raisonnables
        predicted_winrate = max(0.0, min(predicted_winrate, 1.0))
        
        return predicted_winrate, coherence_desc
    
    def predict_winner(self, lineup1: List[Dict], lineup2: List[Dict]) -> Tuple[List[Dict], float, float, str, str]:
        """Prédit le gagnant entre deux lineups"""
        score1, desc1 = self.calculate_lineup_score(lineup1)
        score2, desc2 = self.calculate_lineup_score(lineup2)
        
        if score1 > score2:
            return lineup1, score1, score2, desc1, desc2
        else:
            return lineup2, score2, score1, desc2, desc1

# Exemple d'utilisation
if __name__ == "__main__":
    predictor = LineupPredictor()
    predictor.load_data()
    
    # Sélectionner deux lineups aléatoires
    lineup1 = predictor.select_random_lineup()
    lineup2 = predictor.select_random_lineup()
    
    # Prédire le gagnant
    winner, winner_score, loser_score, winner_desc, loser_desc = predictor.predict_winner(lineup1, lineup2)
    
    print("Lineup 1:")
    for player in lineup1:
        print(f"{player['position']}: {player['name']} ({player['team']})")
        if player['combine_data']:
            print(f"  Combine: {player['combine_data']}")
    print(f"Winrate prédit: {winner_score if winner == lineup1 else loser_score:.3f}")
    print(f"Description: {winner_desc if winner == lineup1 else loser_desc}")
    
    print("\nLineup 2:")
    for player in lineup2:
        print(f"{player['position']}: {player['name']} ({player['team']})")
        if player['combine_data']:
            print(f"  Combine: {player['combine_data']}")
    print(f"Winrate prédit: {winner_score if winner == lineup2 else loser_score:.3f}")
    print(f"Description: {winner_desc if winner == lineup2 else loser_desc}")
    
    print("\nGagnant prédit:")
    for player in winner:
        print(f"{player['position']}: {player['name']} ({player['team']})")
        if player['combine_data']:
            print(f"  Combine: {player['combine_data']}")
    print(f"Winrate prédit: {winner_score:.3f}")
    print(f"Description: {winner_desc}") 

/var/folders/t6/j8g_7ls97b9gdfqfkc7p75q80000gn/T/ipykernel_64614/1388680659.py:145: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data = self.players_data.groupby(['season_year', 'team_abbreviation']).apply(


Lineup 1:
nan: Chuck Hayes (LAC)
nan: Anfernee Hardaway (PHX)
G: Jeremy Lin (BKN)
nan: Ryan Bowen (OKC)
nan: Malik Newman (CLE)
  Combine: {'height': np.float64(189.23), 'wingspan': np.float64(196.85), 'standing_reach': np.float64(250.19), 'body_fat': np.float64(6.25), 'vertical_leap': np.float64(27.5), 'max_vertical': np.float64(33.5), 'agility': np.float64(10.9), 'sprint': np.float64(3.13)}
Winrate prédit: 0.630
Description: Lineup avec une seule position

Lineup 2:
G: Jamal Crawford (BKN)
nan: Henry Ellenson (TOR)
nan: Mac McClung (PHI)
F: Josh Powell (HOU)
nan: Ryan Anderson (HOU)
Winrate prédit: 0.703
Description: Lineup avec Guards et Forwards

Gagnant prédit:
G: Jamal Crawford (BKN)
nan: Henry Ellenson (TOR)
nan: Mac McClung (PHI)
F: Josh Powell (HOU)
nan: Ryan Anderson (HOU)
Winrate prédit: 0.703
Description: Lineup avec Guards et Forwards


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
